In [6]:
# Load required functions GithubCmd
import json

import requests
import requests_mock
from io import BytesIO


class RequestHttp:
    def __init__(
        self,
        http_adapter: requests_mock.Adapter = None,
    ):
        self.http_session = requests.Session()
        if http_adapter:
            self.http_session.mount("https://", http_adapter)

    def set_http_adapter(self, http_adapter: requests_mock.Adapter):
        self.http_session.mount("https://", http_adapter)



class GithubCmd(RequestHttp):
    def __init__(
        self,
        github_token: str,
    ):
        self.github_token = github_token
        super().__init__()

    def get_github_user_data(
        self,
    ):
        url: str = "https://api.github.com/user"
        headers: dict[str, str] = {
            "Accept": "application/vnd.github+json",
            "Authorization": f"Bearer {self.github_token}",
            "X-GitHub-Api-Version": "2022-11-28",
        }

        data = self.http_session.get(
            url=url,
            headers=headers,
        )

        return json.loads(data.text)


class FreshdeskCmd(RequestHttp):
    def __init__(
        self,
        subdomain: str,
        freshdesk_token: str,
    ):
        self.subdomain = subdomain
        self.freshdesk_token = freshdesk_token
        super().__init__()

    def serialize_contact_from_git(
        self,
        user_data,
    ):
        contact_keys: dict[str, str] = {
            "name": "name",
            "email": "email",
            "location": "address",
            "bio": "description",
        }
        serialized_contact = dict()
        for git_key, fresh_key in contact_keys.items():
            serialized_contact[fresh_key] = user_data.get(git_key)

        return serialized_contact, user_data.get("avatar_url", "")

    def update_contact(
        self,
        user_id: int,
        contact_data,
    ):
        url: str = f"https://{self.subdomain}.freshdesk.com/api/v2/contacts/{user_id}"
        headers: dict[str, str] = {
            "Content-Type": "application/json",
        }

        response_obj = self.http_session.put(
            url=url,
            json=contact_data,
            auth=(self.freshdesk_token, "X"),
            headers=headers,
        )
        
        if response_obj.status_code == 404:
            raise LookupError(
                f"User with email {contact_data.get('email')} already exist but is not a contact"
            )

        return json.loads(response_obj.text)

    def create_contact(
        self,
        contact_data,
    ):
        url: str = f"https://{self.subdomain}.freshdesk.com/api/v2/contacts"
        headers: dict[str:str] = {
            "Content-Type": "application/json",
        }

        response_obj = self.http_session.post(
            url=url,
            json=contact_data,
            auth=(self.freshdesk_token, "X"),
            headers=headers,
        )

        # Case Duplicate a unique value
        if response_obj.status_code == 409:
            print(
                f"user with email {contact_data.get('email')} already registered"
            )
            response_conflic = json.loads(response_obj.text,)
            # Get first error value
            error_detail: FreshContactCreateDetailError = response_conflic.get("errors",)[0]
            user_id: int = error_detail.get("additional_info").get("user_id")

            result = self.update_contact(
                user_id=user_id,
                contact_data=contact_data,
            )
            print(f"Contact {contact_data.get('email')} updated")
            return result

        print(f"Contact {contact_data.get('email')} created")
        return json.loads(response_obj.text)

    def list_all_contacts(
        self,
    ):
        url: str = f"https://{self.subdomain}.freshdesk.com/api/v2/contacts"
        headers: dict[str:str] = {
            "Content-Type": "application/json",
        }

        response_obj = self.http_session.get(
            url=url,
            auth=(self.freshdesk_token, "X"),
            headers=headers,
        )
        print(response_obj.text)

print("OK")
    

OK


In [7]:
# Main Run
GITHUB_TOKEN=""
FRESHDESK_SUBDOMAIN=""
FRESHDESK_TOKEN=""

github_cmd = GithubCmd(github_token=GITHUB_TOKEN)
freshdesk_cmd = FreshdeskCmd(
    subdomain=FRESHDESK_SUBDOMAIN,
    freshdesk_token=FRESHDESK_TOKEN,
)

user_data = github_cmd.get_github_user_data()
contact_data, avatar_url = freshdesk_cmd.serialize_contact_from_git(
    user_data=user_data,
)

fresh_result = freshdesk_cmd.create_contact(
    contact_data=contact_data,
)

print(fresh_result)

Contact example_random@gmail.com created
{'active': False, 'address': 'Colombia, Risaralda, Pereira', 'deleted': False, 'description': 'Versatile and skilled in software development with over five years of experience in various roles, led teams, requirements, software planning, DevOps and more.', 'email': 'example_random@gmail.com', 'id': 151033650655, 'job_title': None, 'language': 'en', 'mobile': None, 'name': 'Carlos Arturo', 'phone': None, 'time_zone': 'Eastern Time (US & Canada)', 'twitter_id': None, 'custom_fields': {}, 'tags': [], 'other_emails': [], 'facebook_id': None, 'created_at': '2023-06-12T17:37:26Z', 'updated_at': '2023-06-12T17:37:26Z', 'csat_rating': None, 'preferred_source': None, 'company_id': None, 'view_all_tickets': None, 'other_companies': [], 'unique_external_id': None, 'avatar': None, 'first_name': 'Carlos', 'last_name': 'Arturo', 'visitor_id': 'da27c277-e7be-4bda-af61-70c990bcc8fc', 'org_contact_id': 1668311559356354560, 'other_phone_numbers': []}
